In [1]:
import psycopg2
from psycopg2.extras import execute_values
from bs4 import BeautifulSoup
from multiprocessing import Pool

In [2]:
connection = psycopg2.connect(user="postgres", password="postgres", host="127.0.0.1", port="63333", database="stack_overflow")

In [3]:
with connection:
    with connection.cursor() as cursor:
        cursor.execute("""select title, body
        from posts_19
        limit 5""")
        posts = cursor.fetchall()

In [4]:
posts

[('laravel : symlink storage in public not working " 404 (not found)"',
  '<p>when I try to visualise my picture with this basic url on google :</p>\n<blockquote>\n<p>domain.com/storage/transports/picture.png</p>\n</blockquote>\n<p>I get <code>(404) not found</code>.</p>\n<p>(I don\'t have this error for a file outside the storage symlink in my tree picture and I can totaly see it without any problems, exemple :</p>\n<blockquote>\n<p>domain.com/logo.svg</p>\n</blockquote>\n<p>I have used <code>php artisan storage:link</code> to create the symlink, so far I\'m abble to store the pictures exactly where I want to with that controller :</p>\n<pre><code>public function storeTransports(Request $request)\n    {\n        try{\n            session([\'content\'=&gt;\'content_transports\']);\n\n            $request-&gt;validate([\n                \'name\' =&gt; [ \'string\', \'max:255\'],\n                \'picture\' =&gt; [ \'required\',\'image\',\'mimes:jpeg,png,jpg,gif,svg\', \'max:2048\']\n  

In [5]:
soup = BeautifulSoup(posts[0][1])
print(soup.get_text())

when I try to visualise my picture with this basic url on google :

domain.com/storage/transports/picture.png

I get (404) not found.
(I don't have this error for a file outside the storage symlink in my tree picture and I can totaly see it without any problems, exemple :

domain.com/logo.svg

I have used php artisan storage:link to create the symlink, so far I'm abble to store the pictures exactly where I want to with that controller :
public function storeTransports(Request $request)
    {
        try{
            session(['content'=>'content_transports']);

            $request->validate([
                'name' => [ 'string', 'max:255'],
                'picture' => [ 'required','image','mimes:jpeg,png,jpg,gif,svg', 'max:2048']
            ]);

            if(Transport::where('name', $request->name)->first()){
                return redirect(RouteServiceProvider::HOME)->with('error_transports', 'Transport already exists');
            }
            else{
                if($request

# Extract and transform to text all posts

In [6]:
with connection:
    with connection.cursor() as cursor:
        cursor.execute("""SELECT count(*)
            FROM information_schema.columns
            WHERE table_name = 'posts_19'
            AND column_name = 'body_text'
        """)
        if cursor.fetchall()[0][0] == 0:
            cursor.execute("ALTER TABLE posts_19 ADD COLUMN body_text TEXT;")
            print("Column created")
        else:
            print("Column already exists")

Column already exists


In [28]:
with connection:
    with connection.cursor() as cursor:
        cursor.execute("""select count(*)
        from posts_19 where body is not null""")
        total_posts = cursor.fetchall()[0][0]
total_posts

17069451

In [29]:
def extract_text(id_body):
    post_id, body = id_body
    return post_id, BeautifulSoup(body, 'html.parser').get_text()

In [ ]:
batch_size = 300000
total_batches = (total_posts//batch_size)+1

for batch_index in range(total_batches):

    with connection:
        with connection.cursor() as cursor:
            cursor.execute(f"""select id, body
            from posts_19
            where body is not null
            order by id
            limit {batch_size} offset {batch_size*batch_index}""")
            posts = cursor.fetchall()
            
    with Pool(8) as p:
        maped_posts = p.map(extract_text, posts)
    
    with connection:
        with connection.cursor() as cursor:
            execute_values(cursor, """UPDATE posts_19 SET body_text = data.body_text
                                    FROM (VALUES %s) AS data (id, body_text)
                                    WHERE posts_19.id = data.id""",
                                    maped_posts)
    print(f"{(batch_index+1)/total_batches*100:.2f}% posts procesados")

In [20]:
with connection:
    with connection.cursor() as cursor:
        cursor.execute("select count(*) from posts_19 where body_text is not null;")
        posts_body_sample = cursor.fetchall()[0][0]
posts_body_sample

[(1755,)]

In [21]:
with connection:
    with connection.cursor() as cursor:
        cursor.execute("select id, body, body_text from posts_19 where body_text is not null limit 3;")
        posts_body_sample = cursor.fetchall()
posts_body_sample

[(28756,
  '<p>Whats the best/easiest way to obtain a count of items within an IEnumerable collection without enumerating over all of the items in the collection?</p>\n\n<p>Possible with LINQ or Lambda?</p>\n',
  'Whats the best/easiest way to obtain a count of items within an IEnumerable collection without enumerating over all of the items in the collection?\nPossible with LINQ or Lambda?\n'),
 (16747,
  '<p>I am building a public website using ASP.NET, as part of the deliverable I need to do an Admin Site for data entry of the stuff shown in the public site, I was wondering what techniques or procedures are people using to validate entries using ASP.NET MVC.</p>\n',
  'I am building a public website using ASP.NET, as part of the deliverable I need to do an Admin Site for data entry of the stuff shown in the public site, I was wondering what techniques or procedures are people using to validate entries using ASP.NET MVC.\n'),
 (22776,
  '<p>One more, using <a href="http://docs.python.